Batuhan Uzun - 070170075
Özge Yılmaz - 070180029
Şükran Han - 070180128
Taha Galata - 070190002
Teoman İnan - 070180033

In [2]:
import requests
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd

In [3]:
category_list = ["ACTION", "ARCADE", "PUZZLE", "EDUCATIONAL", "CASUAL", "CARD", "WORD", "CASINO", "ADVENTURE", "BOARD", "MUSIC", "ROLE_PLAYING", "SIMULATION", "TRIVIA", "SPORTS", "STRATEGY", "RACING"]
category_pages = []

for category in category_list:
    category_pages.append('https://play.google.com/store/apps/category/GAME_'+category)
    
games_link_list = []

In [4]:
page_list = category_pages.copy()

page_list.append("https://play.google.com/store/apps/category/GAME")
page_list.append("https://play.google.com/store/apps/top/category/GAME")
page_list.append("https://play.google.com/store/apps/new/category/GAME")

for page in category_pages:  
    category_page = requests.get(page).text
    page_soup = BeautifulSoup(category_page, 'html.parser')
    page_a_list = page_soup.findAll("a", {"class":"id-track-click"})
    for a in page_a_list:
        page_list.append("https://play.google.com"+a["href"])


        
for page in page_list:
    game_page = requests.get(page).text
    game_soup = BeautifulSoup(game_page, 'html.parser')
    game_a_list = game_soup.select("div.b8cIId.ReQCgd.Q9MA7b a")

    for a in game_a_list:
        link = "https://play.google.com"+a["href"]
        if link not in games_link_list:
            games_link_list.append(link)

In [82]:
columns = ["Name", "Category", "Icon", "Video", "Description", "Size", "Content_Rating", "Purchase_Range", "Interactive_Elements", "Downloads", "Score", "Offered_By"]
data = pd.DataFrame(columns=columns)

In [ ]:
for num, game in enumerate(games_link_list):
    game_dict = {"Name":None, "Category":None, "Icon":None, "Video":None, "Description":None, "Size":None, "Content_Rating":None, "Purchase_Range":None, "Interactive_Elements":None, "Downloads":None, "Score":None, "Offered_By":None}
    # Get the game page
    game_page = requests.get(game).text
    game_soup = BeautifulSoup(game_page, 'html.parser')

    try:
        # Name
        game_name = game_soup.find("h1").find("span").text
        game_dict["Name"]= game_name

        # Category
        category = game_soup.find("a", {"itemprop":"genre"}).text
        game_dict["Category"] = category
        print("\r",num, "  -  ", category,"  -  ", game_name, end='')

        # Icon
        image_link = game_soup.find("img", {"class":"T75of sHb2Xb"})["src"]
        img_data = requests.get(image_link).content
        icon_name = game_name.replace(' ', '_')+"_icon"
        icon_name = icon_name.replace(':', '')  
        with open(f"./icons/{icon_name}.jpg", 'wb') as handler:
          handler.write(img_data)
        game_dict["Icon"] = icon_name

        # Video
        video_link = game_soup.find("img", {"class":"T75of DYfLw"})["src"]
        video_data = requests.get(video_link).content
        video_name = game_name.replace(' ', '_')+"_video"
        video_name = video_name.replace(':', '')  
        with open(f"./videos/{video_name}.jpg", 'wb') as handler:
            handler.write(video_data)
        game_dict["Video"] = video_name
        
        # Info In Divs
        divs = game_soup.findAll("div", {"class":"BgcNfc"})
        div_texts = [x.text for x in divs]
        
        # Description
        game_dict["Description"] = game_soup.find("div", {"jsname":"sngebd"}).text

        # Size
        try:
            idx = div_texts.index("Size")
            target_div = divs[idx]
            game_dict["Size"] = target_div.parent.find("span", {"class":"htlgb"}).text
        except:
            pass
        
        #Contect Rating
        try:
            idx = div_texts.index("Content Rating")
            target_div = divs[idx]
            game_dict["Content_Rating"] = target_div.parent.find("span", {"class":"htlgb"}).find("div").find("span").find("div").text
        except:
            pass

        #Purchase_Range
        try:
            idx = div_texts.index("In-app Products")
            target_div = divs[idx]
            game_dict["Purchase_Range"] = target_div.parent.find("span", {"class":"htlgb"}).text
        except:
            pass

        #Interactive_Elements
        try:
            idx = div_texts.index("Interactive Elements")
            target_div = divs[idx]
            game_dict["Interactive_Elements"] = target_div.parent.find("span", {"class":"htlgb"}).text
        except:
            pass

        #Downloads
        try:
            idx = div_texts.index("Installs")
            target_div = divs[idx]
            game_dict["Downloads"] = target_div.parent.find("span", {"class":"htlgb"}).text
        except:
            pass

        #Score
        try:
            game_dict["Score"] = game_soup.find("div", {"class":"BHMmbe"}).text
        except:
            pass
        
        #Offered_By
        idx = div_texts.index("Offered By")
        target_div = divs[idx]
        game_dict["Offered_By"] = target_div.parent.find("span", {"class":"htlgb"}).text

    except:
        continue

    # Append to DataFrame
    data = data.append(game_dict, ignore_index=True)

data.to_csv("data.csv", index=False)

In [144]:
data.groupby("Category").count()["Name"]

Category
Action          219
Adventure       193
Arcade          130
Board           101
Card             54
Casino           77
Casual          210
Educational     166
Music            64
Puzzle          184
Racing          109
Role Playing    239
Simulation      231
Sports          144
Strategy        222
Trivia           25
Word            138
Name: Name, dtype: int64

In [145]:
data

,Name,Category,Icon,Video,Description,Size,Content_Rating,Purchase_Range,Interactive_Elements,Downloads,Score,Offered_By
0,Crossy Road,Action,Crossy_Road_icon,Crossy_Road_video,Join over 200 MILLION players worldwide and ex...,84M,Everyone,$0.99 - $29.99 per item,Digital Purchases,"100,000,000+",4.6,HIPSTER WHALE
1,Cover Fire: Offline Shooting,Action,Cover_Fire_Offline_Shooting_icon,Cover_Fire_Offline_Shooting_video,"Download Official Cover Fire, is one of the be...",47M,Teen,$0.99 - $84.99 per item,In-Game Purchases,"100,000,000+",4.7,Viva Games Studios
2,Temple Run 2,Action,Temple_Run_2_icon,Temple_Run_2_video,"With over a zillion downloads, Temple Run rede...",126M,Everyone,$0.99 - $49.99 per item,In-Game Purchases,"500,000,000+",4.5,Imangi Studios
3,Gangstar Vegas: World of Crime,Action,Gangstar_Vegas_World_of_Crime_icon,Gangstar_Vegas_World_of_Crime_video,Gangstar Vegas is a role-playing game to becom...,51M,Mature 17+,$0.99 - $99.99 per item,In-Game Purchases,"100,000,000+",4.3,Gameloft SE
4,Mario Kart Tour,Action,Mario_Kart_Tour_icon,Mario_Kart_Tour_video,■Challenge players worldwide in multiplayer!Yo...,140M,Everyone,$1.99 - $69.99 per item,In-Game Purchases,"100,000,000+",4.3,"Nintendo Co., Ltd."
...,...,...,...,...,...,...,...,...,...,...,...,...
2507,Motorsport Manager Mobile 2,Racing,Motorsport_Manager_Mobile_2_icon,Motorsport_Manager_Mobile_2_video,The long-awaited sequel to the highest-rated m...,43M,Teen,$2.49 - $11.99 per item,NaN,"100,000+",4.5,Playsport Games
2508,Mini Motor Racing,Racing,Mini_Motor_Racing_icon,Mini_Motor_Racing_video,"Little Cars, BIG Fun!Start your engines! The m...",30M,Everyone,$0.99 - $14.99 per item,Digital Purchases,"100,000+",4.3,The Binary Mill
2509,Dirt Trackin 2,Racing,Dirt_Trackin_2_icon,Dirt_Trackin_2_video,We’ve overhauled the engine to get you the lat...,169M,Everyone,$0.99 per item,NaN,"10,000+",4.6,"Bennett Racing Simulations, LLC"
2510,PJ Masks™: Racing Heroes,Racing,PJ_Masks™_Racing_Heroes_icon,PJ_Masks™_Racing_Heroes_video,"Race with Catboy, Owlette and Gekko on this th...",37M,Everyone,NaN,NaN,"1,000,000+",4.1,Entertainment One
